In [1]:
import numpy as np
import scipy.ndimage as ft
import rasterio
#import dataset.syn_dataset_gen as data_gen
#from metrics import Q2n
import multiprocessing as mp


from math import ceil, floor, log2, sqrt
from skimage.transform.integral import integral_image as integral

import torch
from torch import nn

import dask
import dask.array as da
from dask.distributed import LocalCluster, Client

In [3]:
import numpy as np

def pad_to_compute(labels, q_block_size = 32, q_shift = 32):
    height, width, depth = labels.shape
    stepx = ceil(height / q_shift)
    stepy = ceil(width / q_shift)

    if stepy <= 0:
        stepx = 1
        stepy = 1

    est1 = (stepx - 1) * q_shift + q_block_size - height
    est2 = (stepy - 1) * q_shift + q_block_size - width

    if (est1 != 0) and (est2 != 0):
        labels = np.pad(labels, ((0, est1), (0, est2), (0, 0)), mode='edge')
        labels = labels.astype(labels.dtype)

    height, width, depth = labels.shape

    if ceil(log2(depth)) - log2(depth) != 0:
        exp_difference = 2 ** (ceil(log2(depth))) - depth
        diff_zeros = np.zeros((height, width, exp_difference), dtype="float64")
        labels = np.concatenate([labels, diff_zeros], axis=-1)
        
    return labels

def pad_to_power_of_2_channels(img):
    """
    Pad the channel dimension to the nearest power of 2.
    Required for Cayley-Dickson construction in Q2n.
    
    Parameters:
        img (np.ndarray): Image. Shape: (H, W, C)
        
    Returns:
        img (np.ndarray): Padded image. Shape: (H, W, C') where C' is power of 2
    """
    height, width, depth = img.shape
    
    if ceil(log2(depth)) - log2(depth) != 0:
        exp_difference = 2 ** ceil(log2(depth)) - depth
        diff_zeros = np.zeros((height, width, exp_difference), dtype=img.dtype)
        img = np.concatenate([img, diff_zeros], axis=-1)
    
    return img


def pad_for_sliding_window(img, block_size, shift):
    """
    Pad image to ensure complete coverage by sliding windows.
    
    Parameters:
        img (np.ndarray): Image. Shape: (H, W, C)
        block_size (int): Window size
        shift (int): Stride
        
    Returns:
        img (np.ndarray): Padded image
    """
    height, width, depth = img.shape
    
    stepx = ceil(height / shift)
    stepy = ceil(width / shift)
    
    est1 = (stepx - 1) * shift + block_size - height
    est2 = (stepy - 1) * shift + block_size - width
    
    if est1 > 0 or est2 > 0:
        pad_h = max(0, est1)
        pad_w = max(0, est2)
        img = np.pad(img, ((0, pad_h), (0, pad_w), (0, 0)), mode='reflect')
    
    return img

In [6]:
x = np.random.rand(1024, 1024, 180)
y = pad_to_compute(x)
print(x.shape, y.shape)

(1024, 1024, 180) (1024, 1024, 256)


In [7]:
y1 = pad_for_sliding_window(x, 32, 32)
y2 = pad_to_power_of_2_channels(y1)
print(y2.shape)
 

(1024, 1024, 256)


In [11]:
np.sum(y2!=y)

0

In [8]:
import numpy as np
def preprocess_for_metrics(sharp, pan, hs, ratio=6):
    """
    Preprocess images to ensure dimension compatibility.
    
    This ensures that:
    1. HR dimensions are divisible by ratio
    2. HR dimensions / ratio == LR dimensions
    
    Args:
        sharp: Sharpened image (H, W, C)
        pan: Panchromatic (H, W, 1)
        hs: Hyperspectral (H/ratio, W/ratio, C)
        ratio: Resolution ratio
        
    Returns:
        tuple: (sharp_padded, pan_padded, hs_padded)
    """
    
    hr_h, hr_w = sharp.shape[:2]
    lr_h, lr_w = hs.shape[:2]
    
    # Step 1: Ensure HR dimensions are divisible by ratio
    # This is CRITICAL for D_s metric
    pad_h = (ratio - (hr_h % ratio)) % ratio
    pad_w = (ratio - (hr_w % ratio)) % ratio
    
    if pad_h > 0 or pad_w > 0:
        sharp = np.pad(sharp, ((0, pad_h), (0, pad_w), (0, 0)), mode='edge')
        pan = np.pad(pan, ((0, pad_h), (0, pad_w), (0, 0)), mode='edge')
    
    # Step 2: Ensure LR dimensions match HR/ratio
    expected_lr_h = sharp.shape[0] // ratio
    expected_lr_w = sharp.shape[1] // ratio
    
    lr_pad_h = expected_lr_h - lr_h
    lr_pad_w = expected_lr_w - lr_w
    
    if lr_pad_h > 0 or lr_pad_w > 0:
        hs = np.pad(hs, ((0, lr_pad_h), (0, lr_pad_w), (0, 0)), mode='edge')
    
    # Verify alignment
    assert sharp.shape[0] == pan.shape[0]
    assert sharp.shape[1] == pan.shape[1]
    assert sharp.shape[0] // ratio == hs.shape[0]
    assert sharp.shape[1] // ratio == hs.shape[1]
    
    return sharp, pan, hs

In [9]:

s = np.random.rand(3072, 3072, 180).astype(np.float32)
p = np.random.rand(3072, 3072, 1).astype(np.float32)
h = np.random.rand(510, 510, 180).astype(np.float32)

sharp, pan, hs = preprocess_for_metrics(s, p, h, ratio=6)

In [8]:
print(sharp.shape, pan.shape, hs.shape)
print(s.shape, p.shape, h.shape)

(3072, 3072, 180) (3072, 3072, 1) (512, 512, 180)
(3072, 3072, 180) (3072, 3072, 1) (510, 510, 180)


In [2]:
# test_small.py
import numpy as np
from pansharpening_metrics import compute_metrics, MetricsConfig, preprocess_for_metrics

# Create small test data
sharp = np.random.rand(192, 192, 180).astype(np.float32)
pan = np.random.rand(192, 192, 1).astype(np.float32)
hs = np.random.rand(32, 32, 180).astype(np.float32)

# Compute
config = MetricsConfig.custom()  # Fast for testing
metrics = compute_metrics(sharp, pan, hs, ratio=6, config=config, sensor="PRISMA")

# Check
print(f"D_lambda: {metrics['D_lambda']:.4f}")
print(f"D_s: {metrics['D_s']:.4f}")
print(f"HQNR: {metrics['HQNR']:.4f}")

assert 0 <= metrics['D_lambda'] <= 1
assert 0 <= metrics['D_s'] <= 1
assert 0 <= metrics['HQNR'] <= 1

print("✅ Small example works!")

Debug --- sharp: (192, 192, 180) - pan: (192, 192, 1) - hs: (32, 32, 180)
Computing D_s with 14 workers...
Debug --- SHAPE PAN PRIMA DI RESIZE_PAN: (192, 192, 1) 
Debug --- NUOVO SHAPE PAN DOPO RESIZE_PAN: (192, 192, 1) 
Debug --- NUOVO SHAPE PAN DOPO EXPAND_DIMS: (192, 192, 1) 


c:\Users\tooRi\anaconda3\envs\segmentation\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\tooRi\anaconda3\envs\segmentation\Lib\site-packages\distributed\client.py:3370: UserWarning: Sending large graph of size 26.18 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Computing D_lambda with 14 workers...
D_lambda: 0.9951
D_s: 0.0057
HQNR: 0.0049
✅ Small example works!
